# Imports

In [1]:
# Python Script
%run -i "Scripts/cleaning_data.py"

[INFO] Running Label Encoding...
[INFO] Running text vectorizer...


In [2]:
# Neural Networks
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D, LSTM
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import AveragePooling1D, AveragePooling2D, AveragePooling3D, MaxPooling1D

Using TensorFlow backend.


# Building Convolutional Neural Network

### Preparing

### Reshape Dataset

Conv1D expects an input shape as a 3D:

[batch_size, time_steps, input_dimension]

However, current data is in a shape of:

[batch_size, features]

In [3]:
# Reshaping data
sample_size = X_train.shape[0]
time_steps = X_train.shape[1]
input_dimension = 1

X_train_reshape = X_train.reshape(sample_size, time_steps, input_dimension)
y_train_reshape = np.expand_dims(y_train, axis=1)

# X_test_reshape = X_train_array.reshape(sample_size, time_steps, input_dimension)
# y_test_reshape = np.expand_dims(y_train_array, axis=1)

# X_val_reshape = X_train_array.reshape(sample_size, time_steps, input_dimension)
# y_val_reshape = np.expand_dims(y_train_array, axis=1)

In [4]:
X_test_reshape = X_test.reshape(y_test.shape[0], y_test.shape[1], 1)
y_test_reshape = np.expand_dims(y_test, axis=1)

# X_val_reshape = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_val_reshape = np.expand_dims(X_val, axis=2)
y_val_reshape = np.expand_dims(y_val, axis=1)

In [5]:
X_train_reshape.shape

(24000, 1, 1)

In [6]:
# Instantiating Keras model
cnn_model = Sequential()

# model.add(Flatten())  # Reducing dimensions

# Building Layers
cnn_model.add(Conv1D(100, 1, 
          activation="relu", 
          input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2])))

cnn_model.add(Dense(50, activation="relu"))

cnn_model.add(Conv1D(50, 1, activation="relu"))
cnn_model.add(Conv1D(25, 1, activation="relu"))
cnn_model.add(Conv1D(12, 1, activation="relu"))
cnn_model.add(Conv1D(6, 1, activation="relu"))

cnn_model.add(Dense(3, activation="relu"))
cnn_model.add(Dense(1, activation="relu"))

cnn_model.add(Conv1D(1, 1, activation="softmax"))

print(cnn_model.summary())
# Producing Neural Network
cnn_model.compile(loss='binary_crossentropy',  # Loss function
            optimizer="sgd",  # Gradient Descent
            metrics=['acc'])  # Accuracy score

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 100)            200       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 50)             5050      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 50)             2550      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 25)             1275      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1, 12)             312       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1, 6)              78        
_________________________________________________________________
dense_2 (Dense)              (None, 1, 3)             

In [7]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.1, patience=5,
    verbose=0, mode='auto', min_lr=0)

In [8]:
# If there are problems with input data then it will stop it
nan_problem = keras.callbacks.TerminateOnNaN()

In [9]:
# Stops Epochs when modeling stops to improve
early_stop = keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=5,
    verbose=0, mode='auto')

In [ ]:
cnn1 = cnn_model.fit(X_train_reshape,
               y_train_reshape,
               epochs=80,
               batch_size=10,
               callbacks=[early_stop, nan_problem, reduce_lr])

In [21]:
X_train_reshape

array([[[array([0., 0., 0., ..., 0., 0., 0.])]],

       [[array([0., 0., 0., ..., 0., 0., 0.])]],

       [[array([0., 0., 0., ..., 0., 0., 0.])]],

       ...,

       [[array([0., 0., 0., ..., 0., 0., 0.])]],

       [[array([0., 0., 0., ..., 0., 0., 0.])]],

       [[array([0., 0., 0., ..., 0., 0., 0.])]]], dtype=object)

In [ ]:
X

# Building a Short Long Term Neural Network

In [12]:
lstm_model = Sequential()

lstm_model.add(LSTM(100, return_sequences=True, 
                input_shape=(X_train_reshape.shape[1], X_train_reshape.shape[2]), 
                activation="relu"))

lstm_model.add(Dropout(0.3))

lstm_model.add(Dense(50, activation="relu"))

lstm_model.add(LSTM(50, return_sequences=True, activation="relu"))
lstm_model.add(Dropout(0.3))

lstm_model.add(Dense(25, activation="relu"))

lstm_model.add(LSTM(25, return_sequences=True, activation="relu"))
lstm_model.add(Dropout(0.3))

lstm_model.add(Dense(10, activation="relu"))
lstm_model.add(Dense(10, activation="softmax"))

lstm_model.add(LSTM(1, return_sequences=True, activation="relu"))

lstm_model.summary()

# Producing Neural Network
lstm_model.compile(loss='binary_crossentropy',  # Loss function
            optimizer="sgd",  # Gradient Descent
            metrics=['acc'])  # Accuracy score

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 100)            40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 100)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1, 50)             5050      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1, 25)             1275      
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 25)            

In [13]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.1, patience=5,
    verbose=0, mode='auto', min_lr=0)

In [14]:
# If there are problems with input data then it will stop it
nan_problem = keras.callbacks.TerminateOnNaN()

In [15]:
# Stops Epochs when modeling stops to improve
early_stop = keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=5,
    verbose=0, mode='auto')

In [16]:
lstm1 = lstm_model.fit(X_train_reshape,
               y_train_reshape,
               epochs=80,
               batch_size=10,
               callbacks=[early_stop, nan_problem, reduce_lr])

Epoch 1/80


ValueError: setting an array element with a sequence.